# Overview
1. Data Collection: We use text of Shakespeare's Hamlet as our dataset.
2. Preprocessing: The text data is tokenized, converted into sequences ,and padded to ensure uniform input lengths. The sequences are then split into training and testing sets.
3. Model-Building: An LSTM model is constructed with an embedding layer, two LSTM layers, and a dense output layer with a softmax activation function to predict the probability of the next word.
4. Model Training: The model is trained using prepared sequences, with early stopping implemented to prevent overfitting. Early stopping moniters the validation loss and stops training when loss stops inproving.
5. Model Evaluation: The model is evaluated using a set of example sentences to test its ability to predict the next word accurately
6. Deployement using Streamlit. 

In [38]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

#load the dataset
data = gutenberg.raw('shakespeare-hamlet.txt')
with open('hamlet.txt','w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [39]:
#data preprocessing
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

with open('hamlet.txt') as file:
    text = file.read().lower()

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index)+1
total_words

4818

In [40]:
#create input sequences
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [41]:
max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len


14

In [42]:
input_sequences = np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [43]:
#create predicitors and label
import tensorflow as tf
x,y = input_sequences[:,:-1],input_sequences[:,-1]

In [44]:
y = tf.keras.utils.to_categorical(y,num_classes=total_words)
y


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [45]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [46]:
# Train LSTM RNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,GRU

model = Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
model.add(GRU(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(100))
model.add(Dense(total_words,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

c:\Users\HP\OneDrive\Desktop\ML\venv\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [47]:
history = model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test))

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - accuracy: 0.0326 - loss: 7.1970 - val_accuracy: 0.0348 - val_loss: 6.7567
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - accuracy: 0.0399 - loss: 6.4555 - val_accuracy: 0.0490 - val_loss: 6.7652
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.0566 - loss: 6.1753 - val_accuracy: 0.0565 - val_loss: 6.7405
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.0692 - loss: 5.8558 - val_accuracy: 0.0676 - val_loss: 6.7988
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.0854 - loss: 5.5695 - val_accuracy: 0.0744 - val_loss: 6.8674
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 17s 27ms/step - accuracy: 0.0991 - loss: 5.2622 - val_accuracy: 0.0727 - val_loss: 6.9229
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 17s 27ms/step - accuracy: 0.1107 - loss: 5.0106 - val_accuracy: 0.0750 - val_loss: 6.9886
Epoch 8/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step - accuracy: 0.1300 - loss: 4.7633 - 

In [48]:
def predict_next_word(model,tokenizer,text,max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list)>=max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):] #ensure the sequence length matches max_sequence length
    token_list = pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
    predicted = model.predict(token_list,verbose=0)
    predicted_word_index = np.argmax(predicted,axis=1)
    for word,index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None


In [50]:
input_text = "To be or to not to"
print(f"Input text: {input_text}")
next_word = predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next Word Prediction: {next_word}")

Input text: To be or to not to
Next Word Prediction: this


In [51]:
model.save("next_word_lstm.h5")
import pickle

with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)